# Downward Camera Recording + Line Detection
In this document, you will learn about how to record the output of the downward-facing camera and use it in your code development for line following.


## `rosbag` Usage

`rosbag` is an ROS tool that allows you to record the output of any number of ROS topics while the drone is in operation. This tool is useful for code development because it means we can test without having a drone flying around all the time. We will be using it to look at the LED strip you will have to follow in this week's challenge. 

### Syntax
`rosbag record [options] /<topicname> [/<moretopics>...]`
`[options]` should be replaced with `-O <name>.bag` in most cases.

Topic of interest for line following: `/aero_downward_camera/image`


### Setting up for `rosbag`
SSH into the drone, then:
```
sudo -E bwsi-uav/catkin_ws/src/aero-optical-flow/build/aero-optical-flow
cd ~/rosbags
``` 
The first command will run optical flow


(if ~/rosbags doesn't exist, `mkdir ~/rosbags` and then `cd ~/rosbags`)

Now, you will record the bag file. Make sure to use `rqt_image_view` over SSH (`ssh -Y` enables this behavior) and **check that the downward camera is on and over the area of interest before you start recording!** Also, bag files can get incredibly large when recording images, so limit recordings to ~30 seconds at a time. **We will enforce this.**
```
rosbag record -O downward.bag /aero_downward_camera/image
```

## Post-bag Clean-up
Check disk usage with `sudo df -h /`. The disk might be pretty full at this point. 
We will now transfer the bag file from the drone to your laptop and delete the bag file from the drone.

On your laptop:
```
cd ~/Desktop (or other dir)
scp uav@<teamname>.beaver.works:~/rosbags/downward.bag ~/Desktop/.
```
On drone (thru SSH):
```
cd ~/rosbags
rm downward.bag
```

## Playing your bag file
We will now use `rosbag play` to play back the images you recorded. 


### Syntax
`rosbag play /path/to/<filename>.bag` while `roscore` is running somewhere **ON YOUR  LAPTOP, LOCALLY**. 
This will begin to play the images in real time as they were captured on the drone. You can use `rqt_image_view` to visualize them, and you can use them in whatever line detection code you are working on. Distribute as needed among team members so that you can work at night, if so desired. Images will be published on `/aero_downward_camera/image`, as they were on the drone.

## Integrating `opencv` and ROS images
You will now learn how to use ROS images, like those captured in your newly minted bag file, with `opencv`. The key package you will be using to do so is `cv_bridge`. If you do not have `cv_bridge` installed (which you can check by opening a python shell in Terminal and typing `import cv_bridge`), you can install it using `sudo apt-get install ros-kinetic-cv-bridge`. The following code shows an example class that subscribes to the downward camera topic and creates an `opencv` compatible image. It then "processes" the image and publishes it on a new ROS topic.


```python
import cv2 
from cv_bridge import CvBridge, CvBridgeError
import rospy
from sensor_msgs.msg import Image


_DEBUG = True
class ImageToCV:
    def __init__(self):
        rospy.Subscriber("/aero_downward_camera/image", Image, self.image_cb)
        self.image_pub = rospy.Publisher("/image_to_cv/processed", Image, queue_size=1)
        self.bridge = CvBridge()
        
    def image_cb(self, msg):
        try:
            cv_image = self.bridge.imgmsg_to_cv2(msg, "bgr8")
            
        except CvBridgeError as e:
            print(e)
            
        self.process(cv_image)
        
    def process(self, img):
        # cv2 processing goes here
        
        if _DEBUG:
            try:
                self.image_pub.publish(self.bridge.cv2_to_imgmsg(img, "bgr8"))
            
            except CvBridgeError as e:
                print(e)
        
if __name__ == "__main__":
    rospy.init_node("image_to_cv")
    a = ImageToCV()
    
    rospy.spin()
```

### Visualization
You can use `rqt_image_view` to visualize the resulting images on `/image_to_cv/processed`. 